## documentation

* This presentation: [https://github.com/wolf/re-presentation](https://github.com/wolf/re-presentation)
* Online regex tester and debugger [regular expressions 101](https://regex101.com)
* [re module documentation](https://docs.python.org/3.7/library/re.html)
* third party [regex module homepage](https://bitbucket.org/mrabarnett/mrab-regex)
* by Jeffrey Friedl: ["Mastering Regular Expressions" First Edition](https://www.amazon.com/Mastering-Regular-Expressions-Techniques-Handbooks/dp/1565922573) (covers Python) [Third Edition](https://www.amazon.com/Mastering-Regular-Expressions-Jeffrey-Friedl/dp/0596528124) (does *not* cover Python)

In [1]:
import re

## regular expressions themselves

### what is a regular expression?

A regular expression is a formal description of a set of strings and a well-known collection of rules to decide if any given string is *in* that set.  If a string is in the set, the regular expression is said to *match* that string.  The formal description is often referred to as the *pattern*.  The `re` module has a function to test if a regular expression matches a string: `re.match(pattern, string)`.  It actually has a couple others as well.  The regular expression rules are a domain-specific micro language that lives entirely within Python strings.

This document is about regular expressions (sometimes called regexes) and the `re` module from Python 3.7.  Most of it applies to earlier versions of Python but your mileage may vary.

### where will you use regular expressions?

* In your favorite editor for find and replace
* In Python code (or any other language, really)
* In `grep`, `sed`, and `awk`
* When writing compilers, so with `flex`, `lex`, and `re2c`
* In full-text database searches
* When trying to understand other people's code
* To impress your friends at parties

In [2]:
from termcolor import colored

def demo(pattern, match_against, fn=None, show_location=False):
    """demo is built around re.match, but applied to a list of strings
       instead of a single string; and it prints a message for each, appropriate
       to a demonstration.  It uses colors and bold to make things nice.
       It's really *mostly* about printing.
    """
    compiled_pattern = re.compile(pattern)  # don't really need to do this, but it's good form
    if isinstance(match_against, str):  # OK, it takes a list, but if you give it a single string it still works
        match_against = [ match_against ]
    if fn is None:
        fn = re.match
    else:
        show_location = True
        print(f'using re.{fn.__name__}, the pattern ', end='')

    print(colored(f'{pattern}', 'blue', 'on_yellow', attrs=['bold']))
    for string in match_against:
        match = fn(pattern=compiled_pattern, string=string)
        if match:
            location_info = ''
            r = match.span()
            if show_location:
                location_info = f'at {r[0]}-{r[1]} of '
            print(f'  matches {location_info}', end='')
            
            print(colored(f'{string[:r[0]]}', 'green'), end='')
            print(colored(f'{string[r[0]:r[1]]}', 'green', attrs=['bold']), end='')
            print(colored(f'{string[r[1]:]}', 'green'))
        else:
            print(f'  does *not* match ', end='')
            print(colored(f'{string}', 'red'))
    print()

As we progress, the patterns will look more and more complicated and perhaps even cryptic.  Don't be intimidated!

In [3]:
# Simple characters like 'c', 'a', or 't', match themselves
# Note that I specified the pattern as a raw string... make that a habit for yourself
if re.match(pattern=r'cat', string='cat'):
    print('The pattern "cat" matches the string "cat"')

demo(pattern=r'cat', match_against='dog')

The pattern "cat" matches the string "cat"
cat
  does *not* match dog



`re.match` actually only tests that the string *starts* with a match.  Later we'll talk about `re.search` and `re.fullmatch`.

In [4]:
demo(pattern=r'cat', match_against='catch')

cat
  matches catch



#### character classes or sets

In [5]:
# Using a simple set: [aou]
demo(pattern=r'c[aou]t', match_against=['cat', 'cote', 'cute', 'city'])

c[aou]t
  matches cat
  matches cote
  matches cute
  does *not* match city



In [6]:
# Inverting a set: [^y]
demo(r'c[^y]t', ['cat', 'cote', 'cytoplasm', 'cctv'])

c[^y]t
  matches cat
  matches cote
  does *not* match cytoplasm
  matches cctv



In [7]:
# Using ranges: [a-f0-9]
demo(r'0x[a-f0-9]', ['0x2', '0xc', '0xq'])

0x[a-f0-9]
  matches 0x2
  matches 0xc
  does *not* match 0xq



In [8]:
# Using shortcut: \d == [0-9]; . == [^\n]
demo(r'\d\d\d-\d\d\d\d', ['555-12q2', '411', '867-5309'])
demo(r'\d\d\d-\d\d.\d', ['555-12q2', '411', '867-5309'])

\d\d\d-\d\d\d\d
  does *not* match 555-12q2
  does *not* match 411
  matches 867-5309

\d\d\d-\d\d.\d
  matches 555-12q2
  does *not* match 411
  matches 867-5309



#### repetition

In [9]:
# {min, max}, note that \w == [a-zA-Z0-9_] (at least for ASCII)
demo(r'\d{3}-\d{4}', ['555-12q2', '411', '867-5309'])
demo(r'=\w{3,5}=', ['=cat=', '=catch=', '=dogs=', '=catchy=', '=my='])

\d{3}-\d{4}
  does *not* match 555-12q2
  does *not* match 411
  matches 867-5309

=\w{3,5}=
  matches =cat=
  matches =catch=
  matches =dogs=
  does *not* match =catchy=
  does *not* match =my=



In [10]:
# shortcuts for repetition
# ?, +, *, Note that \s is whitespace
# ? == {0,1}, + == {1,}, * == {0,}
demo(r'\d+\s*-?\s*\d*', ['555-\t1212', '411', '12345-abc'], show_location=True)

\d+\s*-?\s*\d*
  matches at 0-9 of 555-	1212
  matches at 0-3 of 411
  matches at 0-6 of 12345-abc



#### alternation

In [11]:
# binds lower than other operators, so cat|dog does what it looks like
demo(r'cat|dog', ['catchy', 'dog-lover', 'apple pie'])

cat|dog
  matches catchy
  matches dog-lover
  does *not* match apple pie



#### groups

In [12]:
# the default kind of group, capturing, is specified with simple parens
demo(r'\d{3}(-\d{4})?', ['555-1212', '411'])

# When you've captured a group, you can reference it later with a backslash
demo(r'(\w+) \1', ['abc def', 'abc abc'])
demo(r'href=([\'\"]).*?\1', ['href="hello"', 'href=\'goodbye\'', 'href=\'abc"'])
# note the non-greedy '.*?' -- that gets the shortest matching string instead of the longest

\d{3}(-\d{4})?
  matches 555-1212
  matches 411

(\w+) \1
  does *not* match abc def
  matches abc abc

href=([\'\"]).*?\1
  matches href="hello"
  matches href='goodbye'
  does *not* match href='abc"



In [13]:
# special groups start with a '?' just inside the left paren
# (?:...) is a non-capturing group
demo(r'(?:\d+-)?(\d{4})', ['555-1212', '4501'])

# 'x' is the verbose flag, this is a flag-setting group
# verbose mode lets me break apart the regex, even onto multiple lines
demo(r'''(?x)  # verbose mode
    \d{3}          # prefix
    ( - \d{4} )?   # optional''', ['555-1212', '411'])

(?:\d+-)?(\d{4})
  matches 555-1212
  matches 4501

(?x)  # verbose mode
    \d{3}          # prefix
    ( - \d{4} )?   # optional
  matches 555-1212
  matches 411



In [14]:
# setting flags for the whole pattern, for just a portion of the pattern
# the 'i' flag makes the pattern case-insensitive
demo(r'(?i)abcXYZ', ['abcxyz', 'AbCXYz'])
demo(r'(?i:abc)XYZ', ['abcxyz', 'abcXYZ', 'ABCXYZ'])

(?i)abcXYZ
  matches abcxyz
  matches AbCXYz

(?i:abc)XYZ
  does *not* match abcxyz
  matches abcXYZ
  matches ABCXYZ



In [15]:
# assertions, positive and negative look-ahead (specialized groups)
# `re.match`: the pattern must match at the beginning of the string;
# `re.search`: the pattern can match anywhere inside the string
demo(r'Isaac(?=\s+Asimov)', ['Isaac Newton', 'Isaac Asimov'], show_location=True)
demo(r'Isaac(?!\s+Newton)', ['Isaac Newton', 'Isaac Asimov'], show_location=True)
demo(r'(?<!particle)-physics', ['nuclear-physics', 'particle-physics'], fn=re.search)
demo(r'(?<=nuclear)-physics', ['nuclear-physics', 'particle-physics'], fn=re.search)

Isaac(?=\s+Asimov)
  does *not* match Isaac Newton
  matches at 0-5 of Isaac Asimov

Isaac(?!\s+Newton)
  does *not* match Isaac Newton
  matches at 0-5 of Isaac Asimov

using re.search, the pattern (?<!particle)-physics
  matches at 7-15 of nuclear-physics
  does *not* match particle-physics

using re.search, the pattern (?<=nuclear)-physics
  matches at 7-15 of nuclear-physics
  does *not* match particle-physics



#### special matches

Look-ahead and look-behind are a special form of match instruction called an 'assertion'.  They match zero characters, but still influence where the actual match can happen.  Here are a couple of other zero-length assertions:

In [16]:
# beginning or end of a word, or not (zero-length)
demo(r'cat\b', ['cat', 'catchy', 'cat-call'], show_location=True)
demo(r'cat\B', ['cat', 'catchy', 'cat-call'], show_location=True)

# beginning or end of the string (zero-length)
demo(r'^abc', ['abcb', 'bbabc'], fn=re.search)
demo(r'abc$', ['abcb', 'bbabc'], fn=re.search)

cat\b
  matches at 0-3 of cat
  does *not* match catchy
  matches at 0-3 of cat-call

cat\B
  does *not* match cat
  matches at 0-3 of catchy
  does *not* match cat-call

using re.search, the pattern ^abc
  matches at 0-3 of abcb
  does *not* match bbabc

using re.search, the pattern abc$
  does *not* match abcb
  matches at 2-5 of bbabc



When the string to search contains newlines, you use the flag `re.MULTILINE`.  Then `^` and `$` match at the beginning and end of a line (any line), and you'll use `\A` to match at the beginning of the entire string, and `\Z` to match at the end.

## the `re` module

We've already discussed pattern strings and the strings against which we are matching.  Let's talk about the other nouns in the `re` world: compiled patterns, match objects, and the `re` module itself.  To get a compiled pattern, call `re.compile(pattern_string, flags=0)`.  A compiled pattern has method calls that are mirrored in the `re` module:

* `match`, `search`, `fullmatch`
* `split`
* `findall`, `finditer`
* `sub`, `subn`

The `re` versions take a compiled pattern *or* a pattern-string; the compiled pattern versions take (optionally) string indeces.  So for example:

`re.match(pattern, string, flags=0)` [doc](https://docs.python.org/3.7/library/re.html#re.match) vs.<br/>
`pattern.match(string[, pos[, endpos]])` [doc](https://docs.python.org/3.7/library/re.html#re.Pattern.match)

In [17]:
# `match` vs. `search` vs. `fullmatch`
# `match` and `search` you know; `fullmatch` must match the entire string
demo(r'cat', ['catchy', 'cat', 'alley-cat'], fn=re.match)
demo(r'cat', ['catchy', 'cat', 'alley-cat'], fn=re.search)
demo(r'cat', ['catchy', 'cat', 'alley-cat'], fn=re.fullmatch)

using re.match, the pattern cat
  matches at 0-3 of catchy
  matches at 0-3 of cat
  does *not* match alley-cat

using re.search, the pattern cat
  matches at 0-3 of catchy
  matches at 0-3 of cat
  matches at 6-9 of alley-cat

using re.fullmatch, the pattern cat
  does *not* match catchy
  matches at 0-3 of cat
  does *not* match alley-cat



In [18]:
# split with a regex can do things regular str.split cannot
row = 'a,  b;\tc;d, e,          f'
re.split(r'[,;]\s*', row)

['a', 'b', 'c', 'd', 'e', 'f']

In [19]:
# or clean the data with re.sub
re.sub(r'[,;]\s*', ', ', row)

'a, b, c, d, e, f'

In [20]:
# just for completeness' sake will look at re.finditer and re.findall
for i, match in enumerate(re.finditer(r'[,;]\s*', row), 1):
    print(f'match {i} spans {match.span()}')

re.findall(r'\w+', row)

match 1 spans (1, 4)
match 2 spans (5, 7)
match 3 spans (8, 9)
match 4 spans (10, 12)
match 5 spans (13, 24)


['a', 'b', 'c', 'd', 'e', 'f']

In [21]:
# you can pass a function as the replacement argument in sub
# that function takes a single argument: the current match object
re.sub(r'\w+', lambda m: m.group().upper(), row)

'A,  B;\tC;D, E,          F'

#### match objects

Matching functions will return a match object (success) or `None` (failure).  A simple `if` is all you need to test for success.  Save the match object to access some details about the match.  For instance, remember we talked about "capturing" groups.  This is where the data in a captured group is saved.

In [22]:
match = re.match(r'(\d{3})-(\d{4})', '867-5309')
if match:
    print(f'group 1: {match.group(1)}')
    print(f'group 2: {match[2]}')
    print(f'try a tuple: {match.group(1, 2)}')
    print(f'group 1 matched the range {match.span(1)}')
    print(f'group 2 matched the range ({match.start(2)}, {match.end(2)})')
    print(f'original pattern: {match.re}')

group 1: 867
group 2: 5309
try a tuple: ('867', '5309')
group 1 matched the range (0, 3)
group 2 matched the range (4, 8)
original pattern: re.compile('(\\d{3})-(\\d{4})')


## tips and tricks

* verbose mode
* to compile or not
* `re.escape`
* `match.expand`
* non-greedy (sometimes called "lazy") matching

#### Simple string operations vs. `re` calls

In [23]:
%timeit 'cat' in 'dog-catcher'

37.3 ns ± 6.71 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
%timeit re.search('cat', 'dog-catcher')

695 ns ± 71.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [25]:
%timeit 'catchy'.startswith('cat')

133 ns ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [26]:
%timeit re.match('cat', 'catchy')

687 ns ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
%timeit 'cat' == 'cat'

25.1 ns ± 4.39 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [28]:
%timeit re.fullmatch('cat', 'cat')

760 ns ± 71.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The conclusion is: if your regular expression is a simple string, it's way faster, simpler, and easier reading to use simple string operations.  If it's a file-system path, you probably want to be working mostly with `os.path`, or possibly `glob`.  Although I parse URLs below, you probably *really* want to be using `urllib.parse`.

## common pitfalls

* failure to make your pattern a raw string
* not understanding `re.MULTILINE` (which we'll use below) and `re.DOTALL`
* mixing `str` with `bytes`
* trying to [parse HTML](https://stackoverflow.com/questions/1732348/regex-match-open-tags-except-xhtml-self-contained-tags/1732454#1732454) or balance parens

## Real-World(ish) Examples

### parsing URLs

In [29]:
# (?P<name>...) is a special group: named capturing group
# and we'll use verbose mode to make a hard pattern a little easier
# but really you should be using urllib.parse

url_pattern_str = r'''
    (?P<protocol>  # URL protocol, required and captured
      http s?      # the 's' in 'https' is optional
    )
    ://            # required, but not captured
    (?P<host>      # host, required _and_ captured
      [^/:]+       # ...stops at the first slash or colon
    )
    (?:            # an optional group for the port
      :            # ...so we don't capture the colon
      (?P<port>    # optional (because of the containing group), but captured
          \d+      # the port is all digits
      )
    )?
    (?P<path>      # path, optional but captured
      /[^?]*       # ...stops at the first question mark
    )?
    (?:            # an optional group for the query
      \?           # ...so we don't capture the '?' that starts a query
      (?P<query>
          .+       # everything _after_ the question mark is the query itself
      )
    )?
'''
url_pattern = re.compile(url_pattern_str, re.VERBOSE)

urls = [
    'https://google.com',
    'https://google.com/',
    'https://www.kidsa-z.com/main/Activity/reading',
    'http://local.kidsa-z.com:8088/main/Activity/reading?module=razkids',
    'https://github.com/wolf/re-presentation.git',
    'https://www.amazon.com/Mastering-Regular-Expressions-Techniques-Handbooks/dp/1565922573',
]

for url in urls:
    match = url_pattern.match(url)

    for group_name in url_pattern.groupindex.keys():
        if match[group_name]:
            print(f'{group_name:>8}:', colored(f'{match[group_name]}', 'green'))
    print()

protocol: https
    host: google.com

protocol: https
    host: google.com
    path: /

protocol: https
    host: www.kidsa-z.com
    path: /main/Activity/reading

protocol: http
    host: local.kidsa-z.com
    port: 8088
    path: /main/Activity/reading
   query: module=razkids

protocol: https
    host: github.com
    path: /wolf/re-presentation.git

protocol: https
    host: www.amazon.com
    path: /Mastering-Regular-Expressions-Techniques-Handbooks/dp/1565922573



### extracting flags from a status page (abridged)

In [30]:
import os.path

# I've cleaned partial-web-page-dump.txt of passwords, but I still don't want
# to check it into the repository.  I'll just make this cell take a pass if
# you don't have it.  Which you don't.

if os.path.exists('partial-web-page-dump.txt'):
    with open('partial-web-page-dump.txt', 'r') as f:
        text = f.read()

    enabled_features_pattern = re.compile('enabledFeatures[^\(]+\(([^\)]*)\)', re.MULTILINE)
    enabled_features = enabled_features_pattern.search(text)

    # The switch itself is the thing inside the single quotes
    switch_pattern = re.compile("'(.+?)' => true", re.MULTILINE)
    switches = set(switch_pattern.findall(text, *enabled_features.span()))

    print('\n'.join(sorted(switches)))

ACCESS_CONTROL_USAGE_REPORTING_EMAIL
ACCOUNTS_IMPORT_TEACHERS
ACCOUNTS_REGISTER_USER_CHANGES
ACTIVITY_TESTLET_RENAME
ADDITIONAL_PARENT_LETTER_LANGUAGES
ADMIN_DASHBOARDS
ADMIN_DASHBOARDS_REWRITE
AUTHORING_TOOL
AVATAR_BUILDER_REDESIGN
BATCHADD_SISID
BENCHMARK_BOOK_REVISIONS_PROJECT
BOOK_REVIEW_FAVORITES
BRAINHONEY_LINK_ACCOUNTS
BRAINHONEY_LINK_HEADSPROUT
BRAINHONEY_LINK_KIDSAZ
BRAINHONEY_LINK_RAZ
BRAINHONEY_LINK_RAZKIDS
BRAINHONEY_LINK_SAZ
BRAINHONEY_LINK_VOCAB
BRAINHONEY_LINK_WAZ
CLASSROOM_REPORTS_ANGULAR_UI_ROUTING
CLEAR_SESSION_FOR_SSO_AUTHENTICATION
CMS_STICKER_BOOK
CONSTRUCTED_RESPONSE_TOGGLE
CREATE_FLAG_CATEGORIES
CSI_SOLR_SEARCH
CSI_SSO_AUTHENTICATION
DEMO_ROSTER
DEMO_ROSTER_ACTIVITY
DESK_CUSTOMER_SERVICE
DESK_LIVE_CHAT_SERVICE
DISABLE_ACT_ON_EMAIL_CREATE
DISABLE_ACT_ON_EMAIL_SEND
EL_MULTI_LINEITEM_ORDER_API
EL_SFC_LINE_ITEMS
EL_SFC_TAX
ENABLE_MOST_RECENTLY_AVAILABLE_DIST_RPT_DATE
EUROPEAN_CURRICULUM_STANDARDS
FFMPEG_2_0_2
FF_CREATE_NOTES
FF_HTML5_CLIENTS
FF_NEW_DEFINITIONS
FF_NOT